In [ ]:
import os
import json
import collections
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms

from scipy.interpolate import interp1d

from pymedphys.msq import mosaiq_connect
from pymedphys import Delivery
from pymedphys_databases.msq.delivery import delivery_data_from_mosaiq

In [ ]:
field_id = 77695

trf_file = glob(
    r'S:\Physics\Programming\data\LinacLogFiles\indexed\rccc\00*\clinical\{}_3-1_2ARCVMAT_VMAT\2017-05-15_141343_2694\*.trf'.format(
    field_id))[0]
trf_file

In [ ]:
logfile_delivery = Delivery.from_logfile(trf_file)

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_dd = delivery_data_from_mosaiq(cursor, field_id)

In [ ]:
colour = iter(['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])

fig, ax = plt.subplots(figsize=(7,4))

mlc_num = 48
leaf_bank = 1

jaw = np.array(logfile_delivery.jaw)

leaf_pos = (mlc_num - 40) * 5 - 2.5  # Not sufficiently tested
is_blocked = np.invert((-jaw[:,0] <= leaf_pos) & (jaw[:,1] >= leaf_pos))

ax.plot(mosaiq_dd.monitor_units, np.array(mosaiq_dd.mlc)[:, mlc_num, leaf_bank], next(colour), label='Planned MLC position', alpha=0.8)
ax.plot(logfile_delivery.monitor_units, np.array(logfile_delivery.mlc)[:, mlc_num, leaf_bank], next(colour), label='Logfile reported MLC position', alpha=0.8)

trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
behind_marker = ax.fill_between(
    logfile_delivery.monitor_units, 0, 1, 
    where=is_blocked, facecolor='grey', alpha=0.2, 
    transform=trans, label='Behind diaphragm')

plt.xlabel('Cumulative Monitor Units (MU)')
plt.ylabel('Leaf position (mm)')
plt.title('A planned and reported leaf position during a VMAT treatment')

plt.legend()

plt.xlim([0, logfile_delivery.monitor_units[-1]])
plt.savefig('mlc_deviation.png',dpi=300)